:!cabal install --lib pptable
:!cabal install --lib vector
:!cabal install --lib pretty

In [1]:
:load ParseLoc

### create Location for test

In [2]:
x = Location {database = "ushvr12", dbdesc = "dev", host = "odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com", loc_name = "sedhp", loc_class = "oracle", loc_directory = "/u01/app/oracle/product/19.3.0.0/dbee_1", loc_remote_node = "ora-mke2-scanp.am.health.ge.com", loc_remote_login = "gehc_hvr5", loc_remote_pwd = "", loc_remote_port = 4344, loc_db_name = "", loc_db_user = "PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes", loc_description = "scdhp connection"}

####  Visualize Location

In [3]:
import Data.Text.Prettyprint.Doc

pretty x

Location {
  database = ushvr12
  dbdesc = dev
  host = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_name = sedhp
  loc_class = oracle
  loc_directory = /u01/app/oracle/product/19.3.0.0/dbee_1
  loc_remote_node = ora-mke2-scanp.am.health.ge.com
  loc_remote_login = gehc_hvr5
  loc_remote_pwd = 
  loc_remote_port = 4344
  loc_db_name = 
  loc_db_user = PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes
  loc_description = scdhp connection
}

### load Json

In [4]:
import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Control.Applicative

input <- B.readFile "data2.json"
let mm = decode input :: Maybe [Location]
(length.unjust) mm

2596

In [5]:
l = unjust mm
let l_example =  (l!!2)

In [6]:
pretty l_example

Location {
  database = ushvr12
  dbdesc = dev
  host = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_name = sedhp
  loc_class = oracle
  loc_directory = /u01/app/oracle/product/19.3.0.0/dbee_1
  loc_remote_node = ora-mke2-scanp.am.health.ge.com
  loc_remote_login = gehc_hvr5
  loc_remote_pwd = 
  loc_remote_port = 4344
  loc_db_name = 
  loc_db_user = PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes
  loc_description = scdhp connection
}

----------------

import qualified Data.Text as T (Text,append,pack,unpack,head,take,splitOn,intercalate)
import Data.Maybe (fromMaybe)
import qualified Data.ByteString.Lazy.Char8 as BSL
import qualified Data.Vector as V

--- Redshift parser

parseRedshift :: Location  -> [T.Text]
parseRedshift l  = getUSer l : (map T.pack . concat . filterCols . fromMaybe []. parseJson .  getJsonStr)  l  
   where
       getJsonStr  = BSL.pack. T.unpack .last . T.splitOn "=" . loc_db_name  -- get loc_db_name and return string after first '='
       parseJson s = decode s :: Maybe [[String]]            -- parse to list
       filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
       getUSer     = head . T.splitOn "/" . loc_db_user
--- Kafka parser

getKafkaString   = BSL.pack .T.unpack . last . T.splitOn "=" 

valueFromString :: BSL.ByteString -> Maybe Value
valueFromString  s = decode  s :: Maybe Value

unjust :: Maybe a -> a
unjust (Just a) = a

unArray :: Value -> [Value]
unArray (Array x )  = V.toList x

valueToText :: Value ->  T.Text
valueToText val = case toJSON val of
     String txt ->  txt

listFromArray    = T.intercalate ";" . map valueToText . unArray

unString :: [Value] -> [T.Text]
unString  [String x , String y] = [x,  y]
unString  [String x , Array y]  = [x , listFromArray (Array y) ]

getval :: T.Text -> [[T.Text]] -> T.Text
getval key records =
    case filter (\[k, _] -> k == key) records of
        [] -> ""  -- Return an empty string when no match is found
        [[_, value]] -> value
        _ -> error "Multiple matches found"  -- Handle this case as needed

getvals :: [T.Text] -> [[T.Text]] -> [T.Text]
getvals keys lst  = map (`getval` lst) keys

parseKafkaString =    map (unString . unArray) . unArray . unjust .  valueFromString

--- PARSER 

parseLoc :: Location  -> Location'
parseLoc l 
    | loc_class l == "file" && ((T.head . loc_directory)   l  == '/' )     = file_locdir l
    | loc_class l == "file" && ((T.take 4 . loc_directory) l  == "s3s:" )  = file_s3s l
    | loc_class l == "file" && ((T.take 5 . loc_directory) l  == "sftp:")  = file_sftp l
    | loc_class l `elem` ["mysql","greenplum","postgresql"]                = mysql l 
    | loc_class l == "sqlserver"                                           = sqlserver l
    | loc_class l == "teradata"                                            = teradata l 
    | loc_class l == "salesforce"                                          = salesforce l 
    | loc_class l == "redshift"                                            = redsfhit l  
    | loc_class l == "kafka"                                               = kafka l  
    | otherwise   = loc2Location l ""  ["###","###","###"]                                           
     where
        file_locdir x = loc2Location x  "_locdir"  [loc_remote_login  x ,loc_remote_node x, loc_directory x]
        --
        getBacket  =  head . T.splitOn "/"  . last. T.splitOn "@" .loc_directory
        getPrefix  =  T.intercalate  "/" .tail . T.splitOn "/" . last. T.splitOn "@" .loc_directory
        file_s3s x =  loc2Location x "_s3s" ["###",getBacket x ,getPrefix x]
        --
        getsftpNode   = getBacket
        getsftpFolder =  getPrefix
        file_sftp x   = loc2Location x "_sftp" ["###",getsftpNode x ,getsftpFolder x]
        --
        getNodeMysql  =  head . T.splitOn "~" . loc_db_name
        getInstMysql  =  last . T.splitOn "~" . loc_db_name
        getUserMysql  =  head . T.splitOn "/" . loc_db_user
        mysql x =  loc2Location x ""   [ getUserMysql x , getNodeMysql x, getInstMysql x]
        --
        getNodeMsSql  =  head . T.splitOn "\\" . loc_db_name
        getInstMsSql  =  last . T.splitOn "\\" . loc_db_name
        getUserMsSql  =  getUserMysql
        sqlserver x   = loc2Location x ""   [getUserMsSql x, getNodeMsSql x,getInstMsSql x]
        --
        getUserTd     =  getUserMysql
        teradata x = loc2Location x ""   [getUserTd x, loc_db_name x,getUserTd x]
        --
        getNodeSf     = last .T.splitOn "//" . loc_directory
        getInstSf     = last .T.splitOn "@" . head . T.splitOn "/" . loc_db_user
        getUserSf     = head .T.splitOn "@" . head . T.splitOn "/" . loc_db_user
        salesforce x  = loc2Location x ""   [getUserSf x, getNodeSf x,getInstSf x]
        --
        redsfhit x    = loc2Location x ""  (parseRedshift x)
        --
        parseKafka =  getvals ["ssl_key","urls","dummy"] . parseKafkaString . getKafkaString .loc_db_name
        kafka x    =  loc2Location x ""  (parseKafka x)

import qualified Data.Text as T (Text,append,pack,unpack,head,take,splitOn,intercalate)

In [ ]:
import qualified Data.Text as T ( take )
ff l = loc_class l == "file" && ((T.take 5 . loc_directory) l  == "sftp:")  
f l  = loc_class l `elem` ["kafka"]
ll   = filter  f   l

Line 3: Use ==
Found:
loc_class l `elem` ["kafka"]
Why not:
loc_class l == "kafka"

In [9]:
length ll
k = head  ll
pretty k

22

Location {
  database = ushvr04
  dbdesc = US hvr prod
  host = odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com
  loc_name = kafpr
  loc_class = kafka
  loc_directory = 
  loc_remote_node = 10.242.109.196
  loc_remote_login = hvr
  loc_remote_pwd = !{.wruWtPV}!
  loc_remote_port = 4343
  loc_db_name = JSON=[["urls",["b-1.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094","b-3.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094","b-2.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094"]],["broker_ca","/data/kafka_prod/MSKPOCKey.pem"],["ssl_cert","/data/kafka_prod/kafka_prod/signed-certificate-from-acm"],["ssl_key","/data/kafka_prod/kafka_prod/key.pem"],["ssl_key_pwd","!{KQzkyRXnkSraWtnX}!"]]
  loc_db_user = 
  loc_description = kafka prod
}

import qualified Data.ByteString.Lazy.Char8 as BSL
import qualified Data.Vector as V
import qualified Data.Text as T (Text,append,pack,unpack,head,take,splitOn,intercalate)

getKafkaString   = BSL.pack .T.unpack . last . T.splitOn "=" 

valueFromString :: LBS.ByteString -> Maybe Value
valueFromString  s = decode  s :: Maybe Value

unjust :: Maybe a -> a
unjust (Just a) = a

unArray :: Value -> [Value]
unArray (Array x )  = V.toList x

valueToText :: Value ->  T.Text
valueToText val = case toJSON val of
     String txt ->  txt

listFromArray    = T.intercalate ";" . map valueToText . unArray

unString :: [Value] -> [T.Text]
unString  [String x , String y] = [x,  y]
unString  [String x , Array y]  = [x , listFromArray (Array y) ]

getval :: T.Text -> [[T.Text]] -> T.Text
getval key records =
    case filter (\[k, _] -> k == key) records of
        [] -> ""  -- Return an empty string when no match is found
        [[_, value]] -> value
        _ -> error "Multiple matches found"  -- Handle this case as needed

getvals :: [T.Text] -> [[T.Text]] -> [T.Text]
getvals keys lst  = map (`getval` lst) keys

parseKafkaString =    map (unString . unArray) . unArray . unjust .  valueFromString


parseKafka =  getvals ["ssl_key","urls","dummy"] . parseKafkaString . getKafkaString .loc_db_name
parseKafka  k

In [10]:
(map (pretty.parseLoc) . take 5 ) ll

[Location {
  database         = ushvr04
  dbdesc           = US hvr prod
  host             = odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com
  loc_class        = kafka
   loc_name         = kafpr
   loc_directory    = 
   loc_remote_node  = 10.242.109.196
   loc_remote_login = hvr
   loc_remote_port  = 4343
   loc_db_user      = /data/kafka_prod/kafka_prod/key.pem
   loc_description  = kafka prod
   db_node_name     = b-1.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094;b-3.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094;b-2.odp-us-prod-daas-msk.2mp3k5.c7.kafka.us-east-1.amazonaws.com:9094
   db_instance      = 
},Location {
  database         = ushvr04
  dbdesc           = US hvr prod
  host             = odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com
  loc_class        = kafka
   loc_name         = kafk
   loc_directory    = 
   loc_remote_node  = 10.242.112.153
   loc_remote_login = hvr2
   loc_remote_po

In [11]:
import Data.List (nub)
(nub. map loc_class) l

["redshift","file","oracle","sqlserver","teradata","postgresql","salesforce","mysql","greenplum","kafka"]